# **1. 1000개넘을때까지 URL 가져오기**

- 검색어 설정하여 네이버 뉴스 검색 및 URL 저장

- get_news_urls 정의하여 url scraping할 수 있게 설정

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import logging
import pandas as pd
# 로그 출력을 위해서 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') 

search_query = '기독교' # 키워드 설정 (예시:기독교)
encoded_query = requests.utils.quote(search_query)

news_urls = [] # 뉴스 URL저장을 위한 리스트 생성

# 네이버 뉴스 검색 URL 패턴
base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=' 

def get_news_urls(page): # 각 페이지에서 뉴스 URL을 수집하는 함수
    url = f'{base_url}{encoded_query}&start={page*10+1}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    } # 봇으로 감지되지 않도록 User-Agent 설정
    try:
        response = requests.get(url, headers=headers) # HTTP GET 요청을 보냅니다.
        response.raise_for_status() # 응답이 정상인지 확인합니다.
                
        soup = BeautifulSoup(response.content, 'html.parser') #콘텐츠를 파싱
        links = soup.select('a.info')   # 뉴스 링크 요소 선택
        
        # 뉴스 링크를 리스트에 추가
        if not links: 
            return False  # 더 이상 링크가 없으면 False 반환
        
        for link in links:
            news_url = link['href']
            if 'news.naver.com' in news_url: # 네이버 뉴스 링크인지 확인
                news_urls.append(news_url)
        
        logging.info(f'Successfully fetched page {page}, collected {len(links)} links')
        return True  # 링크가 있으면 True 반환
    except requests.exceptions.RequestException as e:
        logging.error(f'Failed to retrieve page {page}, error: {e}')
        return False  # 오류 발생 시 False 반환

- get_news_urls 함수사용하여 url scraping

In [2]:
# 시작 시간 기록
start_time = time.time()

# 페이지를 반복하여 뉴스 URL을 수집
page = 0
max_pages = 400  # 최대 페이지 수 설정(뉴스가 더이상 존재 하지 않을 수 있으므로 무한루프방지)
while len(news_urls) < 1300 and page < max_pages:
    if not get_news_urls(page):
        break  # 더 이상 링크가 없으면 중지
    page += 1
    time.sleep(2)  # 네이버 서버에 과부하를 주지 않기 위해 딜레이를 추가

unique_news_urls = list(set(news_urls))  # 중복 제거
logging.info(f'Removed duplicates, {len(unique_news_urls)} unique URLs remain')

end_time = time.time() # 종료 시간 기록

# 총 소요 시간 계산
elapsed_time = end_time - start_time

# 결과 출력
logging.info(f'총 {len(unique_news_urls)}개의 뉴스 URL을 수집했습니다.')
logging.info(f'총 소요 시간: {elapsed_time:.2f}초')

url_count = len(unique_news_urls)
logging.info(f'중복 제거 후 {url_count}개의 고유한 URL이 수집되었습니다.') # 중복 제거된 URL 개수 출력

df = pd.DataFrame(unique_news_urls, columns=['URL']) # DataFrame 생성

output_filename = f'{search_query}_urls_{url_count}.csv' # 파일명 생성

df.to_csv(output_filename, index=False) # CSV 파일로 저장

logging.info(f'CSV 파일 "{output_filename}"이 성공적으로 생성되었습니다.')

2024-06-02 19:31:02,261 - INFO - Successfully fetched page 0, collected 14 links
2024-06-02 19:31:04,671 - INFO - Successfully fetched page 1, collected 14 links
2024-06-02 19:31:07,054 - INFO - Successfully fetched page 2, collected 16 links
2024-06-02 19:31:09,449 - INFO - Successfully fetched page 3, collected 15 links
2024-06-02 19:31:11,836 - INFO - Successfully fetched page 4, collected 14 links
2024-06-02 19:31:14,193 - INFO - Successfully fetched page 5, collected 13 links
2024-06-02 19:31:16,553 - INFO - Successfully fetched page 6, collected 12 links
2024-06-02 19:31:18,912 - INFO - Successfully fetched page 7, collected 17 links
2024-06-02 19:31:21,284 - INFO - Successfully fetched page 8, collected 12 links
2024-06-02 19:31:23,667 - INFO - Successfully fetched page 9, collected 17 links
2024-06-02 19:31:26,031 - INFO - Successfully fetched page 10, collected 14 links
2024-06-02 19:31:28,407 - INFO - Successfully fetched page 11, collected 14 links
2024-06-02 19:31:30,810 - 

- selenium으로 content, title 추출

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

df = pd.read_csv(output_filename )
# 1000개까지만 url 가져오기
df = df.head(1000)

# Chrome WebDriver 서비스 생성
service = Service("C:\\chromedriver-win64\\chromedriver.exe")
options = Options()
# options.add_argument("--headless")  # 필요 시 헤드리스 모드 사용
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

# 새로운 데이터를 저장할 리스트
data = []

# 스크래핑 시작 시간 기록
start_time = time.time()

# URL 열에서 URL을 행마다 뽑아서 스크래핑 작업 수행
for index, row in df.iterrows():
    link = row['URL']
    logging.info(f'{index + 1}번째 URL 스크랩 중: {link}')
    time.sleep(2)
    driver.get(link)
    
    try:
        # 제목 추출하기
        title_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'media_end_head_headline'))
        )
        title = title_element.text
        logging.info(f'{index + 1}번째 URL 제목 추출 완료')
        
        # 본문 추출하기
        body_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'newsct_article'))
        )
        body = body_element.text.replace('\n', '')
        logging.info(f'{index + 1}번째 URL 본문 추출 완료')
        
        # 데이터를 리스트에 추가
        data.append({
            'link': link,
            'title': title,
            'contnet': body
        })
    except Exception as e: #except 설정안하면 오류나면 코드가 멈춤
        logging.error(f"{index + 1}번째 URL 스크래핑 중 오류 발생: {link}, 오류: {e}")

driver.quit()

# 스크래핑 종료 시간 기록
end_time = time.time()
elapsed_time = end_time - start_time
logging.info(f"Total elapsed time: {elapsed_time:.2f} seconds")

# DataFrame으로 변환
result_df = pd.DataFrame(data)

# 결과 출력
print(result_df)

###키워드변경
output_filename = f'Selenium_news_{search_query}.csv'

# 결과를 CSV 파일로 저장
result_df.to_csv(output_filename, index=False)


2024-06-02 19:46:00,698 - INFO - 1번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/421/0007533064?sid=102
2024-06-02 19:46:03,517 - INFO - 1번째 URL 제목 추출 완료
2024-06-02 19:46:03,647 - INFO - 1번째 URL 본문 추출 완료
2024-06-02 19:46:03,649 - INFO - 2번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/005/0001698400?sid=103
2024-06-02 19:46:06,129 - INFO - 2번째 URL 제목 추출 완료
2024-06-02 19:46:06,203 - INFO - 2번째 URL 본문 추출 완료
2024-06-02 19:46:06,204 - INFO - 3번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/422/0000658212?sid=102
2024-06-02 19:46:08,737 - INFO - 3번째 URL 제목 추출 완료
2024-06-02 19:46:08,774 - INFO - 3번째 URL 본문 추출 완료
2024-06-02 19:46:08,775 - INFO - 4번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/087/0001018030?sid=102
2024-06-02 19:46:11,167 - INFO - 4번째 URL 제목 추출 완료
2024-06-02 19:46:11,194 - INFO - 4번째 URL 본문 추출 완료
2024-06-02 19:46:11,194 - INFO - 5번째 URL 스크랩 중: https://n.news.naver.com/mnews/article/003/0012283299?sid=104
2024-06-02 19:46:13,515 - INFO - 5번째 URL 제목 추출 완료


                                                  link  \
0    https://n.news.naver.com/mnews/article/421/000...   
1    https://n.news.naver.com/mnews/article/005/000...   
2    https://n.news.naver.com/mnews/article/422/000...   
3    https://n.news.naver.com/mnews/article/087/000...   
4    https://n.news.naver.com/mnews/article/003/001...   
..                                                 ...   
995  https://n.news.naver.com/mnews/article/079/000...   
996  https://n.news.naver.com/mnews/article/003/001...   
997  https://n.news.naver.com/mnews/article/081/000...   
998  https://n.news.naver.com/mnews/article/005/000...   
999  https://n.news.naver.com/mnews/article/032/000...   

                                                title  \
0                "세계기독교 성지 건설하는데 자금부족"…1억 챙긴 80대 '실형'   
1                           [새로 나온 책] 당신의 관계는 안녕하십니까?   
2                     현직 경찰, JMS 정명석 증거인멸 연루 의혹…감사 착수   
3                          [포토뉴스]2024년 춘천기독교연합회 신년교례회   
4                 